# Möbius Plateau Explorer v2 🔍

*Loads your fine‑scan rows, buckets them into plateaus, and gives a tiny widget to apply quick ops (×9, ÷9, ×α⁻¹, …).*

> **How to use in Colab**  
> 1. Open the file in the left sidebar → “Refresh” if you don’t see it.  
> 2. Run the notebook top‑to‑bottom.  
> 3. Toggle plateaus & ops in the widget.  
> 4. Use the final cell to download the patched table as Excel.


In [1]:

import pandas as pd, numpy as np

# Paste / load your rows here  ────────────────────────────────
raw_rows = [
    (64,3.6,0.003942,0.0,0.4117525875906678269),
    (64,3.8,0.003942,0.0,0.4117525875906678269),
    (64,4.0,0.003942,0.0,0.4117525875906678269),
    (64,4.2,0.003942,0.0,0.4117525875906678269),
    (128,3.6,0.003942,0.0,0.4117525875906678271),
    (128,3.8,0.003942,0.0,0.4117525875906678271),
    (128,4.0,0.003942,0.0,0.4117525875906678271),
    (128,4.2,0.003942,0.0,0.4117525875906678271),
]

df = pd.DataFrame(raw_rows, columns=['bits','k','phi','lam','rel_err'])


In [2]:

# Simple plateau tag (jump ≈4e‑5)
eps = 4e-5
df['plateau'] = ((df.rel_err - df.rel_err.min()) // eps).astype(int)
df.groupby('plateau')['rel_err'].agg(['min','max','count'])


,min,max,count
plateau,,,
0,0.411753,0.411753,8


In [3]:

import ipywidgets as W
from IPython.display import display, clear_output

alpha_inv = 137.035999084
plateaus = sorted(df.plateau.unique())
sel = W.SelectMultiple(options=plateaus, value=tuple(plateaus), description='Plateaus')
op  = W.Dropdown(options=['identity','×9','÷9','×alpha_inv','÷alpha_inv'], description='Op')
out = W.Output()

def apply(series):
    if op.value=='×9': return series*9
    if op.value=='÷9': return series/9
    if op.value=='×alpha_inv': return series*alpha_inv
    if op.value=='÷alpha_inv': return series/alpha_inv
    return series

def refresh(*_):
    with out:
        clear_output(wait=True)
        view = df[df.plateau.isin(sel.value)].copy()
        view.rel_err = apply(view.rel_err)
        display(view.reset_index(drop=True))

sel.observe(refresh, 'value'); op.observe(refresh, 'value')
display(W.HBox([sel, op])); display(out); refresh()


Output()

In [4]:

df.to_excel('plateaus_v2.xlsx', index=False)
print('Saved ➜ plateaus_v2.xlsx')


Saved ➜ plateaus_v2.xlsx
